# XGBoost

Importar librerías y modelos 

In [109]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

#### Importar base de datos 

In [110]:
genre_data = pd.read_csv('genres_v2.csv')


C:\Users\A01280544\AppData\Local\Temp\ipykernel_22556\3065201629.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  genre_data = pd.read_csv('genres_v2.csv')


In [111]:
print(genre_data.head())
print(genre_data.shape)

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.831   0.814    2    -7.364     1       0.4200        0.0598   
1         0.719   0.493    8    -7.230     1       0.0794        0.4010   
2         0.850   0.893    5    -4.783     1       0.0623        0.0138   
3         0.476   0.781    0    -4.710     1       0.1030        0.0237   
4         0.798   0.624    2    -7.668     1       0.2930        0.2170   

   instrumentalness  liveness  valence  ...                      id  \
0          0.013400    0.0556   0.3890  ...  2Vc6NJ9PW9gD9q343XFRKx   
1          0.000000    0.1180   0.1240  ...  7pgJBLVz5VmnL7uGHmRj6p   
2          0.000004    0.3720   0.0391  ...  0vSWgAlfpye0WCGeNmuNhy   
3          0.000000    0.1140   0.1750  ...  0VSXnJqQkwuH2ei1nOQ1nu   
4          0.000000    0.1660   0.5910  ...  4jCeguq9rMTlbMmPHuO7S3   

                                    uri  \
0  spotify:track:2Vc6NJ9PW9gD9q343XFRKx   
1  spotify:track:7pgJBLVz5VmnL7uGHmR

In [112]:
#NO utilizar!!! Había errores en el valor 'genre' por lo que fue convertido en string
genre_data['genre'] = genre_data['genre'].astype(str)
genre_data.to_csv('genres_v2.csv', index=False)

In [113]:

genre_data = genre_data.drop(columns=['analysis_url','uri','analysis_url','Unnamed: 0', 'id','song_name','track_href','title','key','type' ])


In [100]:
print(genre_data.head())

   danceability  energy  loudness  mode  speechiness  acousticness  \
0         0.831   0.814    -7.364     1       0.4200        0.0598   
1         0.719   0.493    -7.230     1       0.0794        0.4010   
2         0.850   0.893    -4.783     1       0.0623        0.0138   
3         0.476   0.781    -4.710     1       0.1030        0.0237   
4         0.798   0.624    -7.668     1       0.2930        0.2170   

   instrumentalness  liveness  valence    tempo  duration_ms  time_signature  \
0          0.013400    0.0556   0.3890  156.985       124539               4   
1          0.000000    0.1180   0.1240  115.080       224427               4   
2          0.000004    0.3720   0.0391  218.050        98821               4   
3          0.000000    0.1140   0.1750  186.948       123661               3   
4          0.000000    0.1660   0.5910  147.988       123298               4   

       genre  
0  Dark Trap  
1  Dark Trap  
2  Dark Trap  
3  Dark Trap  
4  Dark Trap  


Separamos nuestras Features X de nuestra label y

In [103]:
X = genre_data.drop(columns=["mode"])
y = genre_data['mode']

In [104]:
dataScaler = StandardScaler()
scaler = dataScaler.fit(X)
X_scaled = scaler.transform(X)
print(X_scaled)

[[ 1.22360786  0.2800791  -0.30551447 ...  0.39887636 -1.22699247
   0.10218413]
 [ 0.50847965 -1.46618532 -0.25995376 ... -1.35856366 -0.25679629
   0.10218413]
 [ 1.34492425  0.70984511  0.57203928 ...  2.95986132 -1.4767873
   0.10218413]
 ...
 [-1.77737659  0.31815963  1.14358824 ...  0.31290208 -0.15582161
   0.10218413]
 [-1.03670809  0.86216724  0.57407931 ...  0.10769616  0.70257007
   0.10218413]
 [-0.70468428  0.99272907  0.20517353 ...  0.31759921 -0.861576
   0.10218413]]


Hacemos el split de los datos para crear Train y Test set

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)#

In [106]:
xgboost = xgb.XGBClassifier(n_estimators=200, max_depth=12, learning_rate=0.1, objective='binary:logistic')
xgboost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [107]:
y_pred3 = xgboost.predict(X_test)

In [108]:
print('Misclassified samples: %d' % (y_test != y_pred3).sum())
print(metrics.classification_report(y_test, y_pred3))
print (roc_auc_score(y_test, y_pred3))

Misclassified samples: 2714
              precision    recall  f1-score   support

           0       0.66      0.59      0.62      3796
           1       0.69      0.75      0.72      4665

    accuracy                           0.68      8461
   macro avg       0.68      0.67      0.67      8461
weighted avg       0.68      0.68      0.68      8461

0.6712015637829406


Cross Validation